# TODO
-

# 1. Settings

## 1.1 User variables

In [1]:
MIN_CHARCOAL = 0.1

In [2]:
NB_RUNS = 10

USE_DISCRETE = True

TARGET = "Picea"

NB_TOP_MODELS = 10
TRAIN_SIZE = 0.8
FOLD_STRATEGY = "kfold"
N_FOLD = 5
FIX_IMBALANCE = True # Good for decision tree
FEATURE_SELECTION = False
N_FEATURES_TO_SELECT = 0.2
REMOVE_MULTICOLLINEARITY = False # Good for ada boost sometime
PREPROCESS = True
TRANSFORMATION = True
TRANSFORMATION_METHOD = 'yeo-johnson'
NORMALIZE = True
NORMALIZE_METHOD = "zscore" #maxabs

PCA = False
POLYNOMIAL_FEATURES = False

SORT_METRIC = "MAE"
OPTIMIZE_METRIC = "MAE"

EXCLUDED_MODELS = ["lar"]#["catboost", "nb", "dummy", "lr", "qda", "knn", "lda", "svm", "ridge"] # ["catboost", "Naive Bayes", "Dummy Classifier", "Logistic Regression", "Quadratic Discriminant Analysis", "K Neighbors Classifier"]

# 1.2 Dev variables

In [3]:
RANDOM_SEED = 0

INPUT_DATA = "tmp/data_processed.csv"
INPUT_DATA_DISCRETE = "tmp/data_discrete.csv"

OUTPUT_SCORES_PATH = "tmp/hap_pycaret_scores.csv"
VERBOSE = True

## 1.3 Imports

In [4]:
import pandas as pd
import numpy as np
import random
from pycaret.regression import *

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

## 1.4 Constants

In [5]:
# Constants
DEBUG = False

# 2. Data Loading

In [6]:
if USE_DISCRETE:
    df = pd.read_csv(INPUT_DATA_DISCRETE)
else:
    df = pd.read_csv(INPUT_DATA)

df = df[df[TARGET] >= MIN_CHARCOAL]
display(df)

,M15,Abies,Juniperus,Larix,Picea,Pinus,Tsuga,Acer,Alnus,Betula,...,Myrica,Phytolithe,Poaceae,Rosaceae,Sarcobatus,Thalictrum,Typha,Spore monolete,Spore trilete,Aquatics
0,0,1,1,2,1,1,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,2,2,0
2,0,1,2,2,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0,1,1,2,1,1,0,0,2,1,...,1,0,0,1,0,0,0,0,0,0
4,1,2,0,2,1,1,0,1,0,0,...,1,0,1,0,0,1,0,0,0,0
5,0,2,0,1,1,1,0,0,1,1,...,1,1,0,1,0,1,0,0,0,0
6,0,0,1,2,1,1,0,0,0,0,...,1,1,1,0,0,0,1,1,0,0
7,1,1,0,2,1,1,0,0,0,1,...,2,0,0,0,0,1,1,1,0,1
8,1,1,0,1,1,1,0,0,1,1,...,1,0,1,0,1,0,2,2,0,1
9,0,2,0,2,1,1,0,0,0,2,...,1,0,0,0,0,0,1,0,0,1


# Pycaret AutoML

In [7]:
df_scores = None

setup(df, target = TARGET, session_id = RANDOM_SEED,
    train_size=TRAIN_SIZE,
    fold_strategy=FOLD_STRATEGY,
    fold=N_FOLD,
    #fix_imbalance=FIX_IMBALANCE,
    preprocess=PREPROCESS,
    feature_selection=FEATURE_SELECTION,
    n_features_to_select=N_FEATURES_TO_SELECT,
    remove_multicollinearity=REMOVE_MULTICOLLINEARITY,
    normalize=NORMALIZE,
    normalize_method=NORMALIZE_METHOD,
    pca=PCA,
    polynomial_features=POLYNOMIAL_FEATURES,
    index=False,
    n_jobs=-1
    )

print("Val Scores")
top = compare_models(n_select = NB_TOP_MODELS,
                    sort=SORT_METRIC,
                    exclude=EXCLUDED_MODELS,
                    verbose=VERBOSE,
                    errors="raise")
df_results = pull()
df_results

,Description,Value
0,Session id,0
1,Target,Picea
2,Target type,Regression
3,Original data shape,"(30, 41)"
4,Transformed data shape,"(30, 41)"
5,Transformed train set shape,"(24, 41)"
6,Transformed test set shape,"(6, 41)"
7,Numeric features,40
8,Preprocess,True
9,Imputation type,simple


Val Scores


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.0546,0.0441,0.1216,0.3741,0.0480,0.0481,0.0120
ada,AdaBoost Regressor,0.0800,0.0800,0.1789,0.5500,0.0726,0.0600,0.0080
dt,Decision Tree Regressor,0.1200,0.1200,0.2683,0.3000,0.1089,0.0800,0.0080
gbr,Gradient Boosting Regressor,0.1228,0.0847,0.2492,0.1553,0.1022,0.0985,0.0180
knn,K Neighbors Regressor,0.1300,0.0772,0.2306,-0.0400,0.0977,0.0940,0.0120
rf,Random Forest Regressor,0.1319,0.0710,0.2516,0.1932,0.1041,0.1077,0.0220
et,Extra Trees Regressor,0.1338,0.0946,0.2761,0.0989,0.1125,0.1032,0.0160
lightgbm,Light Gradient Boosting Machine,0.1547,0.0791,0.2326,-0.0543,0.0987,0.1168,0.0100
lasso,Lasso Regression,0.1547,0.0791,0.2326,-0.0543,0.0987,0.1168,0.2060
en,Elastic Net,0.1547,0.0791,0.2326,-0.0543,0.0987,0.1168,0.2140


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.0546,0.0441,0.1216,0.3741,0.0480,0.0481,0.012
ada,AdaBoost Regressor,0.0800,0.0800,0.1789,0.5500,0.0726,0.0600,0.008
dt,Decision Tree Regressor,0.1200,0.1200,0.2683,0.3000,0.1089,0.0800,0.008
gbr,Gradient Boosting Regressor,0.1228,0.0847,0.2492,0.1553,0.1022,0.0985,0.018
knn,K Neighbors Regressor,0.1300,0.0772,0.2306,-0.0400,0.0977,0.0940,0.012
rf,Random Forest Regressor,0.1319,0.0710,0.2516,0.1932,0.1041,0.1077,0.022
et,Extra Trees Regressor,0.1338,0.0946,0.2761,0.0989,0.1125,0.1032,0.016
lightgbm,Light Gradient Boosting Machine,0.1547,0.0791,0.2326,-0.0543,0.0987,0.1168,0.010
lasso,Lasso Regression,0.1547,0.0791,0.2326,-0.0543,0.0987,0.1168,0.206
en,Elastic Net,0.1547,0.0791,0.2326,-0.0543,0.0987,0.1168,0.214


# DEBUG

In [8]:
evaluate_model(top)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

# TODO